In [1]:
pip install datasets

Defaulting to user installation because normal site-packages is not writeable
     |████████████████████████████████| 511 kB 719 kB/s eta 0:00:01
     |████████████████████████████████| 133 kB 3.7 MB/s eta 0:00:01
     |████████████████████████████████| 184 kB 24.2 MB/s eta 0:00:01
     |████████████████████████████████| 11.6 MB 34.2 MB/s eta 0:00:01    |██████▍                         | 2.3 MB 34.2 MB/s eta 0:00:01
     |████████████████████████████████| 200 kB 32.5 MB/s eta 0:00:01
     |████████████████████████████████| 21.2 MB 3.7 MB/s eta 0:00:011   |██                              | 1.4 MB 7.4 MB/s eta 0:00:03     |█████▍                          | 3.6 MB 7.4 MB/s eta 0:00:03     |███████████████████████████████▏| 20.6 MB 3.7 MB/s eta 0:00:01
     |████████████████████████████████| 64 kB 6.4 MB/s  eta 0:00:01
     |████████████████████████████████| 515 kB 58.0 MB/s eta 0:00:01
     |████████████████████████████████| 119 kB 20.9 MB/s eta 0:00:01
  Using cached tqdm-4.67.1-py3-none

In [ ]:
# importing the dataset directly from huggingface
from datasets import load_dataset

# Login using e.g. `huggingface-cli login` to access this dataset
ds = load_dataset("ailsntua/QEvasion")

/Users/saiesh/Library/Python/3.9/lib/python/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/Users/saiesh/Library/Python/3.9/lib/python/site-packages/urllib3/__init__.py:35: NotOpenSSLWarning: urllib3 v2 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(
Generating test split: 100%|██████████| 308/308 [00:00<00:00, 13394.22 examples/s]


In [3]:
ds

DatasetDict({
    train: Dataset({
        features: ['title', 'date', 'president', 'url', 'question_order', 'interview_question', 'interview_answer', 'gpt3.5_summary', 'gpt3.5_prediction', 'question', 'annotator_id', 'annotator1', 'annotator2', 'annotator3', 'inaudible', 'multiple_questions', 'affirmative_questions', 'index', 'clarity_label', 'evasion_label'],
        num_rows: 3448
    })
    test: Dataset({
        features: ['title', 'date', 'president', 'url', 'question_order', 'interview_question', 'interview_answer', 'gpt3.5_summary', 'gpt3.5_prediction', 'question', 'annotator_id', 'annotator1', 'annotator2', 'annotator3', 'inaudible', 'multiple_questions', 'affirmative_questions', 'index', 'clarity_label', 'evasion_label'],
        num_rows: 308
    })
})